<a href="https://colab.research.google.com/github/honoreade/DeepLearning/blob/main/hyperparameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# fruitClassifier Project

In [ ]:
!pip install scikeras
!pip install keras keras-tuner
# Dependency libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
def load_data(data_path):

    # Define image dimensions (adjust as needed)
    img_width, img_height = 224, 224

    # Get the number of classes (assuming subfolders represent classes)
    class_names = os.listdir(data_path)
    num_classes = len(class_names)

    # Initialize empty lists for images and labels
    images = []
    labels = []

    # Loop through each class subfolder
    for class_index, class_name in enumerate(class_names):
        class_path = os.path.join(data_path, class_name)
        # print(class_path)

        # Loop through images in the class subfolder
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            # print(img_path)

            # Load the image using TensorFlow's image module
            img = load_img(img_path, target_size=(img_width, img_height))

            # Convert the image to a NumPy array (optional)
            img_array = img_to_array(img)

            # Preprocess the image (e.g., normalize)
            # ... your preprocessing steps ...

            # Append the image and its class label
            images.append(img_array)
            labels.append(class_index)

    # Convert data to NumPy arrays
    images = np.array(images)
    labels = np.array(labels)

    # One-hot encode labels if needed for categorical classification
    if num_classes > 2:
        labels =  tf.keras.utils.to_categorical(labels, num_classes=num_classes)

    return images, labels

    # Example usage (same as before)

In [ ]:
train_data_path = r"/content/drive/MyDrive/Colab Notebooks/FruitClassifier/FruitClassifier/train"
val_data_path = r"/content/drive/MyDrive/Colab Notebooks/FruitClassifier/FruitClassifier/val"
test_data_path = r"/content/drive/MyDrive/Colab Notebooks/FruitClassifier/FruitClassifier/test"


In [ ]:
train_images, train_labels = load_data(train_data_path)
val_images, val_labels = load_data(val_data_path)
test_images, test_labels = load_data(test_data_path)

**Model Building**

In [ ]:
model = tf.keras.models.Sequential()
# Add layers
model.add(layers.Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(units=32, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Print model summary for debugging
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 224, 224, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 16)        │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 16)          │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 12544)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │         401,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 7)                   │             231 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 406,759 (1.55 MB)

 Trainable params: 406,759 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Define validation data (assuming validation_images and validation_labels exist)
validation_data = (val_images, val_labels)
model.fit(train_images, train_labels, epochs=5, batch_size=32, validation_data=validation_data)

Epoch 1/5
78/78 ━━━━━━━━━━━━━━━━━━━━ 19s 98ms/step - accuracy: 0.1741 - loss: 109.3573 - val_accuracy: 0.0653 - val_loss: 2.1053
Epoch 2/5
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.2839 - loss: 1.7745 - val_accuracy: 0.0528 - val_loss: 2.1766
Epoch 3/5
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.3252 - loss: 1.6737 - val_accuracy: 0.0319 - val_loss: 2.1768
Epoch 4/5
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.3136 - loss: 1.6418 - val_accuracy: 0.0889 - val_loss: 2.6262
Epoch 5/5
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.4158 - loss: 1.4172 - val_accuracy: 0.0875 - val_loss: 3.0801


In [ ]:
# accuracy: 0.1519 - loss: 1.9456 - val_accuracy: 0.1389: single layer
# accuracy: 0.4507 - loss: 1.4275 - val_accuracy: 0.1014: double layer
# accuracy: 0.1535 - loss: 1.9457 - val_accuracy: 0.1389: triple layer


# Hyperparameter tuning

In [ ]:
from keras_tuner import RandomSearch

# Function to build the model with hyperparameters
def model_builder(hp):
    models = tf.keras.models.Sequential()
    models.add(layers.Input(shape=(224, 224, 3)))

    # Tune the number of filters
    for i in range(hp.Int('num_layers', 1, 5)):  # Number of convolutional layers
      filters = hp.Choice('filters' + str(i), values=[32, 64, 128])
      kernel_size = hp.Choice('kernel_size', values=[3, 5])
      models.add(layers.Conv2D(filters, kernel_size, padding='same', activation='relu'))
      models.add(layers.MaxPooling2D(pool_size=(2, 2)))

    models.add(layers.Flatten())
    models.add(layers.Dense(hp.Int('units', 32, 512, step=32), activation='relu'))
    models.add(layers.Dense(7, activation='softmax'))

    models.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0001])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return models

# Instantiate the tuner
tuner = RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=5,  # Number of different hyperparameter combinations to try
    executions_per_trial=3,
    directory='random_search',
    project_name='image_classification'
)


In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': 'linear'}
filters0 (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64, 128], 'ordered': True}
kernel_size (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}


In [ ]:
# Search for the best hyperparameters
tuner.search(train_images, train_labels, epochs=5, validation_data=(val_images, val_labels))

Trial 5 Complete [00h 04m 09s]
val_accuracy: 0.13888888557751974

Best val_accuracy So Far: 0.15787037213643393
Total elapsed time: 00h 40m 16s


In [ ]:
# Get the best hyperparameters after the search
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# Print the best hyperparameters
print("Best Hyperparameters:")
print("Number of CNN layers:", best_hyperparameters.get('num_layers'))

# Loop through each convolutional layer to display its filters and kernel size
for i in range(best_hyperparameters.get('num_layers')):
    # Define keys
    filters_key = f'filters_{i}'
    kernel_size_key = f'kernel_size_{i}'

    # Check if the key exists
    if best_hyperparameters.is_active(filters_key) and best_hyperparameters.is_active(kernel_size_key):
        filters = best_hyperparameters.get(filters_key)
        kernel_size = best_hyperparameters.get(kernel_size_key)

        print(f"Filters for layer {i}: {filters}")
        print(f"Kernel Size for layer {i}: {kernel_size}")
    else:
        print(f"Layer {i} parameters not found")

# Print the best Dense layer configuration
print("Units in Dense layer:", best_hyperparameters.get('units'))

# Print the best learning rate
print("Learning Rate:", best_hyperparameters.get('learning_rate'))


# print("Optimizer:", best_hyperparameters.get('optimizer'))

Best Hyperparameters:
Number of CNN layers: 4
Layer 0 parameters not found
Layer 1 parameters not found
Layer 2 parameters not found
Layer 3 parameters not found
Units in Dense layer: 64
Learning Rate: 0.001


In [ ]:
# Get the best model and evaluate it
best_model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
best_model.fit(train_images, train_labels, epochs=100, validation_data=(val_images, val_labels))

Epoch 1/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - accuracy: 0.7437 - loss: 0.7190 - val_accuracy: 0.1333 - val_loss: 8.3706
Epoch 2/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step - accuracy: 0.7911 - loss: 0.5534 - val_accuracy: 0.1208 - val_loss: 8.9912
Epoch 3/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 74ms/step - accuracy: 0.8067 - loss: 0.6099 - val_accuracy: 0.1056 - val_loss: 9.0691
Epoch 4/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - accuracy: 0.8065 - loss: 0.5390 - val_accuracy: 0.1125 - val_loss: 10.7852
Epoch 5/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 74ms/step - accuracy: 0.8219 - loss: 0.5015 - val_accuracy: 0.1250 - val_loss: 10.9011
Epoch 6/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - accuracy: 0.8308 - loss: 0.4743 - val_accuracy: 0.1167 - val_loss: 13.4060
Epoch 7/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - accuracy: 0.8041 - loss: 0.5120 - val_accuracy: 0.1139 - val_loss: 13.7239
Epoch 8/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 10s 74ms/step - accuracy: 0.8196 - loss: 0.5263 - val_accura

In [ ]:
# Evaluate the model
test_loss, test_accuracy = best_model.evaluate(test_images, test_labels)
print(f"Best Test loss: {test_loss:.4f}, Best Test accuracy: {test_accuracy:.4f}")

44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 66ms/step - accuracy: 0.1427 - loss: 37.7013
Best Test loss: 45.6452, Best Test accuracy: 0.1279


In [ ]:
from keras_tuner import Hyperband


# Function to build the model with hyperparameters
def model_builder(hp):
    models = tf.keras.models.Sequential()
    models.add(layers.Input(shape=(224, 224, 3)))

    # Tune the number of filters
    for i in range(hp.Int('num_layers', 1, 5)):  # Number of convolutional layers
      filters = hp.Choice('filters' + str(i), values=[32, 64, 128])
      kernel_size = hp.Choice('kernel_size', values=[3, 5])
      models.add(layers.Conv2D(filters=filters, kernel_size=(3, 3), activation='relu',
                        kernel_regularizer=tf.keras.regularizers.l2(0.001)))
      models.add(layers.MaxPooling2D(pool_size=(2, 2)))

    models.add(layers.Flatten())
    models.add(layers.Dense(hp.Int('units', 32, 512, step=32), activation='relu'))
    models.add(layers.Dense(7, activation='softmax'))

    models.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0001])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return models

In [ ]:
# Instantiate the Keras Tuner using Hyperband search
tuner = Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=5,
                     factor=3,
                     directory='mydir',
                     project_name='cnn_tuning')


# Search for the best hyperparameter combination
tuner.search(train_images, train_labels, epochs=5, validation_data=(val_images, val_labels))

Trial 10 Complete [00h 00m 47s]
val_accuracy: 0.10277777910232544

Best val_accuracy So Far: 0.14305555820465088
Total elapsed time: 00h 32m 13s


In [ ]:
# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Create a string for optimal filters for each layer
filters_output = ", ".join(
    [f"Layer {i + 1}: {best_hps.get('filters_' + str(i))}" for i in range(best_hps.get('num_layers'))]
)

# Print all the optimal hyperparameters in a single statement
print(f"""
The hyperparameter search is complete.
Optimal number of filters: {filters_output}
Optimal kernel size: {best_hps.get('kernel_size')}
Optimal number of conv layers: {best_hps.get('num_layers')}
Optimal number of dense units: {best_hps.get('units')}
Optimal learning rate: {best_hps.get('learning_rate')}
""")

# # Get the best hyperparameters and model
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# # Print the optimal hyperparameters
# print("The hyperparameter search is complete.")

# # Print the optimal number of conv layers and their filters and kernel sizes
# print(f"Optimal number of conv layers: {best_hps.get('num_layers')}")
# for i in range(best_hps.get('num_layers')):
#     filters = best_hps.get('filters_' + str(i))
#     kernel_size = best_hps.get('kernel_size_' + str(i))
#     print(f" Layer {i + 1}: {filters} filters with kernel size {kernel_size}")

# # Print the optimal dense units and learning rate
# print(f"Optimal dense units: {best_hps.get('units')}")
# print(f"Optimal learning rate: {best_hps.get('learning_rate')}")


The hyperparameter search is complete. 
Optimal number of filters: 
Optimal number of Kernel Size: 5
Optimal number of conv layers: 1
Optimal number of dense units: 96
Optimal learning rate: 0.001



In [ ]:
# Build the best model with optimal hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the best model
history = best_model.fit(train_images, train_labels,
                         epochs=10,
                         validation_data=(val_images, val_labels))


In [ ]:
## General
# import tensorflow as tf
# from tensorflow.keras import layers
# from keras_tuner import RandomSearch

# # Define the model function using HyperParameters from Keras Tuner
# def build_model(hp):
#     model = tf.keras.models.Sequential()
#     filters = hp.Int('filters', min_value=32, max_value=128, step=32)
#     kernel_size = hp.Choice('kernel_size', values=[3, 5])
#     units = hp.Int('units', min_value=64, max_value=256, step=64)

#     model.add(layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu', input_shape=(224, 224, 3)))
#     model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Conv2D(filters=filters * 2, kernel_size=kernel_size, padding='same', activation='relu'))
#     model.add(layers.MaxPooling2D((2, 2)))
#     model.add(layers.Flatten())
#     model.add(layers.Dense(units=units, activation='relu'))
#     model.add(layers.Dense(7, activation='softmax'))

#     optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop'])
#     model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#     return model

# # Create a RandomSearch tuner
# tuner = RandomSearch(
#     build_model,
#     objective='val_accuracy',
#     max_trials=10,
#     executions_per_trial=3,
#     directory='random_search',
#     project_name='image_classification'
# )

# # Prepare your data (replace with your actual data)
# train_data = train_images, train_labels  # Your training data
# val_data = val_images, val_labels    # Your validation data

# # Start the hyperparameter search
# tuner.search(
#     train_data,
#     epochs=20,
#     validation_data=val_data,
#     batch_size=32
# )

# # Retrieve the best model and hyperparameters
# best_model = tuner.get_best_models(num_models=1)[0]
# best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

# print("Best Hyperparameters:")
# print("Filters:", best_hyperparameters.get('filters'))
# print("Kernel Size:", best_hyperparameters.get('kernel_size'))
# print("Units:", best_hyperparameters.get('units'))
# print("Optimizer:", best_hyperparameters.get('optimizer'))


###### End.